In [1]:
import sys
import os
sys.path.insert(0, './../../../DataAndScripts/unstructured_scripts/')

In [2]:
%reset
#%qtconsole
%load_ext autoreload
%autoreload 2
%matplotlib inline 
import matplotlib.pyplot as plt
import functions_old as fun

import numpy as np
import matplotlib as mpl
from scipy import stats

from scipy.interpolate import interp1d,interp2d
from scipy.optimize import least_squares

import pandas as pd





Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Define Network Parameters

In [3]:
sigma_t=0.01; # Input noise, determine how smooth the single neuron transfer function is 
tau_rp=0.002; # Refractory period

g_E=8.; g_I=3;beta=1.; # parameters characterizing the relative strength of synaptic connections
K=500*4; # average number of E connections per neuron (I connections are 1/4 of that)
sigma_Lambda_over_Lambda=1; # ratio between mean and standard deviation of the opsin expression
#sigma_Lambda_over_Lambda=0.5; # ratio between mean and standard deviation of the opsin expression

L=[0.,  20. ]; #  Laser intensity that fits the data
r_X=20*np.ones(1)
#[20,20,20]; # To speed up the code, I show results onlye for low anf high contrast and laser

Tmax_over_tau_E=100;
#Tmax_over_tau_E=15;

p=0.1; # probability of connection between cells. 
sigma_X_over_r_X=0.2; # Presynaptic population has mean rate r_X, Gaussianly distributed with std=0.2* r_X
Lambda=1*10**-3; # mean opsin expression in Volt

sigma_Lambda=sigma_Lambda_over_Lambda*Lambda


N=int(K/p);
N_E,N_I=N,int(N*fun.gamma);
N_X=N_E+N_I

In [4]:
J=0.0006
(sigma_Lambda*L[-1])**2/K/J**2

0.5555555555555557

In [7]:

mu_tab_max=10.0;
mu_tab=np.linspace(-mu_tab_max,mu_tab_max,200000)
mu_tab=np.concatenate(([-10000],mu_tab))
mu_tab=np.concatenate((mu_tab,[10000]))

phi_tab_E,phi_tab_I=mu_tab*0,mu_tab*0;
for idx in range(len(phi_tab_E)):
    phi_tab_E[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_E,tau_rp,sigma_t)
    phi_tab_I[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_I,tau_rp,sigma_t)

phi_int_E=interp1d(mu_tab, phi_tab_E, kind='linear')  
phi_int_I=interp1d(mu_tab, phi_tab_I, kind='linear')


/Users/tuannguyen/Reshuffling/1-MainTextFigures/Figure_4/generate_simulations/./../../../DataAndScripts/unstructured_scripts/functions_old.py:53: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  adelleh=integrate.quad(lambda u: f(u),minimo,massimo)
/Users/tuannguyen/Reshuffling/1-MainTextFigures/Figure_4/generate_simulations/./../../../DataAndScripts/unstructured_scripts/functions_old.py:53: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  adelleh=integrate.quad(lamb

# Simulate Networks

In [8]:
print('Computing and saving network response')
T=np.arange(0,Tmax_over_tau_E*fun.tau_E,fun.tau_I/3);

#mask_time=T>(10*fun.tau_E)
mask_time=T>(T/2)

# Generate quenched disorder
Possible_J=[0.00002,0.0006]#,0.0007,0.0008]#10**np.linspace(-5,-3,20); # Synaptic efficacy in Volts


RATES=-1*np.ones((len(Possible_J),len(L),N_E+N_I))
DYNA=-1*np.ones((len(Possible_J),len(L),N_E+N_I,len(T)))
MUS=-1*np.ones((len(Possible_J),len(L),N_E+N_I))
Lambda_i_L=1./tau_rp*np.ones((len(Possible_J),len(L),N_E+N_I))
MFT_SOL_R=np.ones((len(Possible_J),len(L),10))
MFT_SOL_M=np.ones((len(Possible_J),len(L),10))
Phi=[phi_int_E,phi_int_I];
for idx_J in [0,1]:#range(len(Possible_J)):

    G_E,G_I=1.0,2.0 # Gain of Excitatory and inhibitory cells and I cells
    w_EE=1;w_IE=w_EE/beta;
    w_EI=g_E*w_EE;w_II=g_I*w_IE;
    
    w_EX,w_IX=(G_I*fun.gamma*g_E-G_E)*w_EE,(G_I*fun.gamma*g_I-G_E)*w_IE; 
    J=Possible_J[idx_J]*np.sqrt(500/K)
    if idx_J==0:
        w_X=np.asarray([w_EX,5*w_IX]);
        #w_X=np.asarray([w_EX,w_IX]);
    if idx_J>0:
        w_X=np.asarray([w_EX,w_IX]);
    print(w_X)
    w=np.zeros((2,2));
    w[0,:]=w_EE,-w_EI
    w[1,:]=w_IE,-w_II

    M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I=fun.Generate_quenched_disorder(sigma_X_over_r_X,J,K,w,w_X,p,Lambda,sigma_Lambda)

    print(w*J)
    
    for idx_L in range(len(L)):
        r_X_local,L_local=r_X[0],L[idx_L];
        print((idx_J+1)/len(Possible_J),(idx_L+1)/len(L))
        DYNA[idx_J,idx_L,:,:], MUS[idx_J,idx_L,:],Lambda_i_L[idx_J,idx_L,:]=fun.High_dimensional_dynamics(T,L_local,r_X_local,M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I,phi_int_E,phi_int_I);
        RATES[idx_J,idx_L,:]=np.mean(DYNA[idx_J,idx_L,:,mask_time],axis=0)
        print(np.mean(RATES[idx_J,idx_L,0:N_E]),
              np.mean(RATES[idx_J,idx_L,N_E::]),
              np.std(RATES[idx_J,idx_L,0:N_E]),
              np.std(RATES[idx_J,idx_L,N_E::]))
    Base_Sim=RATES[idx_J,0,:]
    Delta_Sim=RATES[idx_J,idx_L,:]-RATES[idx_J,0,:]

    moments_of_r_sim=np.zeros(5)
    moments_of_r_sim[0]=np.mean(Base_Sim)
    moments_of_r_sim[1]=np.mean(Delta_Sim)
    moments_of_r_sim[2]=np.std(Base_Sim)
    moments_of_r_sim[3]=np.std(Delta_Sim)
    moments_of_r_sim[4]=np.cov(Base_Sim,Delta_Sim)[0,1]
    print(moments_of_r_sim[:])

    

Computing and saving network response
[3.  2.5]
[[ 1.e-05 -8.e-05]
 [ 1.e-05 -3.e-05]]
0.5 0.5
17.183130645265685 23.275535666882263 2.3663801196247047 1.0427117942688515
0.5 1.0
68.04961823933819 51.41855491358233 48.18676301308314 2.750824429861978
[ 18.40161165  46.32179392   3.26129381  43.87095322 -15.97918925]
[3.  0.5]
[[ 0.0003 -0.0024]
 [ 0.0003 -0.0009]]
1.0 0.5


KeyboardInterrupt: 

# Save Parameters, Rates, and Js

In [354]:
params=[sigma_t,tau_rp,g_E,g_I,beta,K,sigma_Lambda_over_Lambda,L[1],r_X[0],Tmax_over_tau_E,p,sigma_X_over_r_X,Lambda,N_E,N_I]

np.save('simulations/RATES',RATES)
np.save('simulations/params',params)
np.save('simulations/Possible_J',Possible_J)


